In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import joblib
import time
import itertools
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
import codecs
from urllib.parse import urljoin
import numpy as np

In [2]:
preflist=[10101,
          10201,10202,10203,10204,10205,10206,
          10301,10302,10303,10304,10305,10306,10307,
          10401,10402,10403,
          10501,10502,10503,
         10601,10602,10603,10604,
         10701,10702,10703,10704,10705,10706,
         10801,10802,10803,10804,10805,
         10901,10902,10903,10904,
         11001,11002,11003,11004,11005,11006,11007,11008
         ,11101]

In [3]:
df = pd.DataFrame(preflist,columns=["都道府県"])
df

,都道府県
0,10101
1,10201
2,10202
3,10203
4,10204
5,10205
6,10206
7,10301
8,10302
9,10303


In [4]:
df['ページ数']=''
for i in range(len(df['都道府県'])):
    url="https://www.ecareer.ne.jp/q/prefecture/{}".format(df['都道府県'][i])
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "lxml")
    divs = soup.select_one('#pageBody > article.searchResult > div:nth-child(2) > ul > li.last > a')
    k=re.findall(r'window=([0-9]{1,5})',str(divs))
    df['ページ数'][i]=k

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
for i in range(len(df['ページ数'])):
    df['ページ数'][i]=max(df['ページ数'][i])
df

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,都道府県,ページ数
0,10101,23
1,10201,16
2,10202,17
3,10203,20
4,10204,16
5,10205,16
6,10206,17
7,10301,19
8,10302,18
9,10303,17


In [7]:
def joblib_url(i):
    url = 0
    new_list = list()
    for z in range(1,int(df['ページ数'][i])):
        urls = 'https://www.ecareer.ne.jp/q/prefecture/{0}?window={1}'.format(df['都道府県'][i],z)
        res = requests.get(urls)
        try:
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "lxml")
            divs = soup.find_all('section',class_='recruitBox')
            for div in divs:
                try:
                    link = div.find('a')
                    href = link.attrs['href'] 
                    new_list.append(href)
                except:
                    pass
        except:
            pass
    return new_list
            

In [8]:
all_list = list()
resultList = joblib.Parallel(n_jobs=-1, verbose=3,require='sharedmem',backend="threading")( [joblib.delayed(joblib_url)(i) for i  in range(len(df['都道府県']))])
all_list.extend(resultList) 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 13.9min finished


In [9]:
new_all_list = list(itertools.chain.from_iterable(all_list))
kekka = pd.DataFrame(new_all_list,columns=["url"])
kekka

,url
0,/positions/00046654002/8
1,/positions/00046654002/9
2,/positions/00021421004/89
3,/positions/00021421004/139
4,/positions/00043737001/4
...,...
28793,/positions/00044070001/4
28794,/positions/00044070001/3
28795,/positions/00000348010/2135
28796,/positions/00000348010/2103


In [208]:
kekka.to_csv('ecareer_url.csv')

In [111]:
k2=kekka.drop_duplicates()
k2=k2.reset_index()
k2

,index,url
0,0,/positions/00046272001/4
1,1,/positions/00047852001/7
2,2,/positions/00047852001/8
3,3,/positions/00047981001/5
4,4,/positions/00021421004/74
...,...,...
6373,28561,/positions/00045655001/1
6374,28579,/positions/00048362001/2
6375,28587,/positions/00031999004/20
6376,28647,/positions/00000348010/2239


In [113]:
url_key=k2['url']

In [208]:
def joblib_get_data(z):
    llist = list()
    url = "https://www.ecareer.ne.jp"+url_key[z]
    r = requests.get(url)
    try:
        r.raise_for_status()
        kk=[]
        soup = BeautifulSoup(r.text, "lxml")
        for i in range(20):
            try:
                k=soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > table:nth-child({}) > tbody > tr:nth-child(1) > td".format(i))
                if k!=None:
                    kk.append(i)
            except:
                pass
        try:
            houzinnmei=soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > div.corpInfo > table > tbody > tr:nth-child(1) > td").get_text()
        except:
            houzinnmei=''
        try:
            sisetumei=""
        except:
            sisetumei=""
        try:    
            honnsya=soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > div.corpInfo > table > tbody > tr:nth-child(4) > td").get_text()
        except:
            honnsya=""
        try:
            page=soup.select_one("#pageBody > section > section.detailBox.job > header > div > div.text > h2").get_text()
        except:
            page=""
        try:    
            kisaisyokusyu=soup.select_one("#pageBody > section > div > dl > dd:nth-child(2) > ul > li:nth-child(3) > a").get_text()
        except:
            kisaisyokusyu=""
        try:
            gyousyu=soup.select_one("#pageBody > section > div > dl > dd:nth-child(4) > ul > li:nth-child(1) > a").get_text()
        except:
            gyousyu=""
        try:
            dennwabangou=soup.select_one("#pageBody > section > section.detailBox.corp > div > div > aside > p:nth-child(4)").get_text()
        except:
            dennwabangou=""
        try:
            sigoto=soup.select_one("#wrapper > div:nth-child(3) > ul > li:nth-child(3) > a").get_text()
        except:
            sigoto=""
        try:
            hakenn=""
        except:
            hakenn=""
        for i in range(20):
            try:
                if soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > table:nth-child({}) > tbody > tr:nth-child({}) > th".format(kk[0],i)).get_text()=='勤務地':
                    kinnmuti=soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > table:nth-child({}) > tbody > tr:nth-child({}) > td".format(kk[0],i)).get_text()
            except:
                pass
        for i in range(20):
            try:                        #pageBody > section > section.detailBox.job > div.detail > div > table:nth-child(10) > tbody > tr:nth-child(11) > td
                if soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > table:nth-child({}) > tbody > tr:nth-child({}) > th".format(kk[0],i)).get_text()=='給与':
                    kyuuyozyouhou=soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > table:nth-child({}) > tbody > tr:nth-child({}) > td".format(kk[0],i)).get_text()
            except:
                pass
        for i in range(20):
            try:
                if soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > table:nth-child({}) > tbody > tr:nth-child({}) > th".format(kk[0],i)).get_text()=='雇用形態':
                    koyoukubun=soup.select_one("#pageBody > section > section.detailBox.job > div.detail > div > table:nth-child({}) > tbody > tr:nth-child({}) > td".format(kk[0],i)).get_text()
            except:
                pass
        llist.append(houzinnmei)#
#        llist.append(sisetumei)#
        llist.append(kinnmuti)#
        llist.append(honnsya)#
        llist.append(koyoukubun)#
        llist.append(page)#
        llist.append(kisaisyokusyu)#
        llist.append(gyousyu)#
        llist.append(dennwabangou)#
        llist.append(kyuuyozyouhou)#
        llist.append(sigoto)#
#        llist.append(hakenn)#
        llist.append(url)#
    except:
        pass
    return llist



url="https://www.ecareer.ne.jp/positions/00021421004/75"
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")
k=soup.select_one('#wrapper > div:nth-child(3) > ul > li:nth-child(3) > a').get_text()
print(k)

In [209]:
all_list = list()
resultList = joblib.Parallel(n_jobs=-1, verbose=3,require='sharedmem',backend="threading")([joblib.delayed(joblib_get_data)(i)  for i in range(100)])
all_list.extend(resultList)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished


In [210]:
df_kekka=pd.DataFrame(all_list,columns=['法人名','勤務地住所','本社住所','雇用区分','ページタイトル','媒体記載職種','業種','電話番号','給与情報','仕事内容','ページurl'])
df_kekka

,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,仕事内容,ページurl
0,株式会社ＥＣＯＰ,\n \n オフィス不要・ご...,\n \n ...,業務委託,窓フィルムの作業スタッフ◆未経験歓迎／切って貼るだけの簡単作業／加盟金・研修費0円／副業OK◆,設備工事スタッフ,清掃・メンテナンス,075-333-4407,\n \n 完全歩合制\n ...,工事スタッフ、その他建築・土木・設備工事関連,https://www.ecareer.ne.jp/positions/00046272001/4
1,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,\n \n ...,正社員,営業職◆全国に勤務地あり／学歴・経験不問《年収4000万も可能》共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,\n\n雇用企業のホームページ\n,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/7
2,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,\n \n ...,正社員,省エネアドバイザー◆学歴・経験不問/年収4000万も可能/共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,\n\n雇用企業のホームページ\n,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/8
3,株式会社ｏｒ ｓｔｙｌｅ,\n \n 勤務地：希望を考...,\n \n ...,正社員,介護に特化した人材サービス営業◆営業・販売経験のある方大歓迎／早期キャリアアップも可能◆,人材コーディネーター,人材派遣・人材紹介・アウトソーシング,06-7777-2615,\n \n ■総合職（転勤の...,人材コーディネーター・カウンセラー,https://www.ecareer.ne.jp/positions/00047981001/5
4,株式会社スタッフサービス エンジニアリング事業本部（リクルートグループ）,\n \n 北海道・東北・北...,\n \n ...,正社員,CADオペレーター◆年間休日125日／残業月平均12時間／チームで助け合えるユニット制あり◆,CADオペレーター（電気・電子・半導体・機械・メカトロ）,その他IT関連,03-5209-7520,\n \n ■月給21万50...,機械・機構・金型・解析,https://www.ecareer.ne.jp/positions/0002142100...
...,...,...,...,...,...,...,...,...,...,...,...
95,野村不動産パートナーズ株式会社,\n \n 全国の当社拠点\...,\n \n ...,正社員,分譲マンションの管理◆野村不動産グループ／残業は20時間前後／年間休日123日◆,不動産管理,不動産仲介・不動産管理,\n\n\n,"\n \n 月給215,00...",不動産関連,https://www.ecareer.ne.jp/positions/00046389001/3
96,株式会社夢真,\n \n 全国いずれかのプ...,\n \n ...,正社員,総合職◆賞与年2回／各種手当充実／年休120日／土日祝休み／面接1回（私服可）／最短5日で内定◆,設備保全,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 月給30万円以上...,施工管理・設備保守管理,https://www.ecareer.ne.jp/positions/00046654003/5
97,株式会社夢真,\n \n 全国いずれかのプ...,\n \n ...,正社員,事務◆志望動機や自己PRは白紙でOK！／最短5日で内定／賞与年2回／年休120日／土日祝休み◆,一般事務・庶務,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 東京・大阪：月給...,一般事務・営業事務,https://www.ecareer.ne.jp/positions/00046654003/3
98,株式会社夢真,\n \n 全国いずれかのプ...,\n \n ...,正社員,点検・保守◆未経験歓迎／賞与年2回／年間休日120日／私服面接可／面接から入社まで最短5日◆,ビル・設備管理,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 東京・大阪：月給...,施設・設備管理・警備・清掃,https://www.ecareer.ne.jp/positions/00046654003/1


In [5]:
df_kekka.to_csv('ecareer_kekka.csv',index=False)

NameError: name 'df_kekka' is not defined

In [10]:
df_ka=pd.read_csv('ecareer_kekka.csv')
df_ka

,Unnamed: 0,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,仕事内容,ページurl
0,0,株式会社ＥＣＯＰ,\n \n オフィス不要・ご...,\n \n ...,業務委託,窓フィルムの作業スタッフ◆未経験歓迎／切って貼るだけの簡単作業／加盟金・研修費0円／副業OK◆,設備工事スタッフ,清掃・メンテナンス,075-333-4407,\n \n 完全歩合制\n ...,工事スタッフ、その他建築・土木・設備工事関連,https://www.ecareer.ne.jp/positions/00046272001/4
1,1,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,\n \n ...,正社員,営業職◆全国に勤務地あり／学歴・経験不問《年収4000万も可能》共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,\n\n雇用企業のホームページ\n,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/7
2,2,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,\n \n ...,正社員,省エネアドバイザー◆学歴・経験不問/年収4000万も可能/共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,\n\n雇用企業のホームページ\n,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/8
3,3,株式会社ｏｒ ｓｔｙｌｅ,\n \n 勤務地：希望を考...,\n \n ...,正社員,介護に特化した人材サービス営業◆営業・販売経験のある方大歓迎／早期キャリアアップも可能◆,人材コーディネーター,人材派遣・人材紹介・アウトソーシング,06-7777-2615,\n \n ■総合職（転勤の...,人材コーディネーター・カウンセラー,https://www.ecareer.ne.jp/positions/00047981001/5
4,4,株式会社スタッフサービス エンジニアリング事業本部（リクルートグループ）,\n \n 北海道・東北・北...,\n \n ...,正社員,CADオペレーター◆年間休日125日／残業月平均12時間／チームで助け合えるユニット制あり◆,CADオペレーター（電気・電子・半導体・機械・メカトロ）,その他IT関連,03-5209-7520,\n \n ■月給21万50...,機械・機構・金型・解析,https://www.ecareer.ne.jp/positions/0002142100...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,野村不動産パートナーズ株式会社,\n \n 全国の当社拠点\...,\n \n ...,正社員,分譲マンションの管理◆野村不動産グループ／残業は20時間前後／年間休日123日◆,不動産管理,不動産仲介・不動産管理,\n\n\n,"\n \n 月給215,00...",不動産関連,https://www.ecareer.ne.jp/positions/00046389001/3
96,96,株式会社夢真,\n \n 全国いずれかのプ...,\n \n ...,正社員,総合職◆賞与年2回／各種手当充実／年休120日／土日祝休み／面接1回（私服可）／最短5日で内定◆,設備保全,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 月給30万円以上...,施工管理・設備保守管理,https://www.ecareer.ne.jp/positions/00046654003/5
97,97,株式会社夢真,\n \n 全国いずれかのプ...,\n \n ...,正社員,事務◆志望動機や自己PRは白紙でOK！／最短5日で内定／賞与年2回／年休120日／土日祝休み◆,一般事務・庶務,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 東京・大阪：月給...,一般事務・営業事務,https://www.ecareer.ne.jp/positions/00046654003/3
98,98,株式会社夢真,\n \n 全国いずれかのプ...,\n \n ...,正社員,点検・保守◆未経験歓迎／賞与年2回／年間休日120日／私服面接可／面接から入社まで最短5日◆,ビル・設備管理,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 東京・大阪：月給...,施設・設備管理・警備・清掃,https://www.ecareer.ne.jp/positions/00046654003/1


In [11]:
df_ka=df_kekka.copy()

NameError: name 'df_kekka' is not defined

In [12]:
df_ka['本社住所']=df_ka['本社住所'].str.replace(" ","").astype(str).str.extract("[0-9]{4}(\D+.+)\n")

In [13]:
df_ka['本社住所']

0                京都府京都市西京区大枝沓掛町2-38
1           福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル
2           福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル
3                  大阪府大阪市西区江戸堀2-1-1
4      東京都千代田区神田練塀町85番地　ＪＥＢＬ秋葉原スクエア
                  ...              
95         東京都新宿区西新宿1-26-2新宿野村ビル21Ｆ
96    東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F
97    東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F
98    東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F
99    東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F
Name: 本社住所, Length: 100, dtype: object

In [14]:
df_ka['都道府県']=''
df_ka['市区町村']=''
for i in range(len(df_ka['本社住所'])):
    df_ka['都道府県'][i]=re.search(r"(大阪府)|(京都府)|(北海道)|(.+県)|(東京都)",df_ka['本社住所'][i])
    df_ka['都道府県'][i]=df_ka['都道府県'][i].group()
    df_ka['市区町村'][i]=re.search(r"(\D+?市)|(\D+?区)|(\D+?町)|(\D+?村)",df_ka['本社住所'][i])
    df_ka['市区町村'][i]=df_ka['市区町村'][i].group()
    df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('京都府','')
    df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('大阪府','')
    df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('北海道','')
    df_ka['市区町村'][i]=re.sub(r'.+県','',df_ka['市区町村'][i])
    df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('東京都','')

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_l

In [15]:
df_ka['都道府県']

0     京都府
1     福岡県
2     福岡県
3     大阪府
4     東京都
     ... 
95    東京都
96    東京都
97    東京都
98    東京都
99    東京都
Name: 都道府県, Length: 100, dtype: object

In [16]:
df_ka['市区町村']

0      京都市
1      福岡市
2      福岡市
3      大阪市
4     千代田区
      ... 
95     新宿区
96    千代田区
97    千代田区
98    千代田区
99    千代田区
Name: 市区町村, Length: 100, dtype: object

In [17]:
df_ka

,Unnamed: 0,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,仕事内容,ページurl,都道府県,市区町村
0,0,株式会社ＥＣＯＰ,\n \n オフィス不要・ご...,京都府京都市西京区大枝沓掛町2-38,業務委託,窓フィルムの作業スタッフ◆未経験歓迎／切って貼るだけの簡単作業／加盟金・研修費0円／副業OK◆,設備工事スタッフ,清掃・メンテナンス,075-333-4407,\n \n 完全歩合制\n ...,工事スタッフ、その他建築・土木・設備工事関連,https://www.ecareer.ne.jp/positions/00046272001/4,京都府,京都市
1,1,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,営業職◆全国に勤務地あり／学歴・経験不問《年収4000万も可能》共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,\n\n雇用企業のホームページ\n,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/7,福岡県,福岡市
2,2,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,省エネアドバイザー◆学歴・経験不問/年収4000万も可能/共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,\n\n雇用企業のホームページ\n,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/8,福岡県,福岡市
3,3,株式会社ｏｒ ｓｔｙｌｅ,\n \n 勤務地：希望を考...,大阪府大阪市西区江戸堀2-1-1,正社員,介護に特化した人材サービス営業◆営業・販売経験のある方大歓迎／早期キャリアアップも可能◆,人材コーディネーター,人材派遣・人材紹介・アウトソーシング,06-7777-2615,\n \n ■総合職（転勤の...,人材コーディネーター・カウンセラー,https://www.ecareer.ne.jp/positions/00047981001/5,大阪府,大阪市
4,4,株式会社スタッフサービス エンジニアリング事業本部（リクルートグループ）,\n \n 北海道・東北・北...,東京都千代田区神田練塀町85番地 ＪＥＢＬ秋葉原スクエア,正社員,CADオペレーター◆年間休日125日／残業月平均12時間／チームで助け合えるユニット制あり◆,CADオペレーター（電気・電子・半導体・機械・メカトロ）,その他IT関連,03-5209-7520,\n \n ■月給21万50...,機械・機構・金型・解析,https://www.ecareer.ne.jp/positions/0002142100...,東京都,千代田区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,野村不動産パートナーズ株式会社,\n \n 全国の当社拠点\...,東京都新宿区西新宿1-26-2新宿野村ビル21Ｆ,正社員,分譲マンションの管理◆野村不動産グループ／残業は20時間前後／年間休日123日◆,不動産管理,不動産仲介・不動産管理,\n\n\n,"\n \n 月給215,00...",不動産関連,https://www.ecareer.ne.jp/positions/00046389001/3,東京都,新宿区
96,96,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,総合職◆賞与年2回／各種手当充実／年休120日／土日祝休み／面接1回（私服可）／最短5日で内定◆,設備保全,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 月給30万円以上...,施工管理・設備保守管理,https://www.ecareer.ne.jp/positions/00046654003/5,東京都,千代田区
97,97,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,事務◆志望動機や自己PRは白紙でOK！／最短5日で内定／賞与年2回／年休120日／土日祝休み◆,一般事務・庶務,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 東京・大阪：月給...,一般事務・営業事務,https://www.ecareer.ne.jp/positions/00046654003/3,東京都,千代田区
98,98,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,点検・保守◆未経験歓迎／賞与年2回／年間休日120日／私服面接可／面接から入社まで最短5日◆,ビル・設備管理,人材派遣・人材紹介・アウトソーシング,\n\n雇用企業のホームページ\n,\n \n 東京・大阪：月給...,施設・設備管理・警備・清掃,https://www.ecareer.ne.jp/positions/00046654003/1,東京都,千代田区


In [18]:
#電話番号
df_ka['電話番号']=df_ka['電話番号'].astype(str).str.extract('([0-9]{2,4}-[0-9]{2,4}-[0-9]{2,4})')

In [19]:
df_ka

,Unnamed: 0,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,仕事内容,ページurl,都道府県,市区町村
0,0,株式会社ＥＣＯＰ,\n \n オフィス不要・ご...,京都府京都市西京区大枝沓掛町2-38,業務委託,窓フィルムの作業スタッフ◆未経験歓迎／切って貼るだけの簡単作業／加盟金・研修費0円／副業OK◆,設備工事スタッフ,清掃・メンテナンス,075-333-4407,\n \n 完全歩合制\n ...,工事スタッフ、その他建築・土木・設備工事関連,https://www.ecareer.ne.jp/positions/00046272001/4,京都府,京都市
1,1,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,営業職◆全国に勤務地あり／学歴・経験不問《年収4000万も可能》共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/7,福岡県,福岡市
2,2,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,省エネアドバイザー◆学歴・経験不問/年収4000万も可能/共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/8,福岡県,福岡市
3,3,株式会社ｏｒ ｓｔｙｌｅ,\n \n 勤務地：希望を考...,大阪府大阪市西区江戸堀2-1-1,正社員,介護に特化した人材サービス営業◆営業・販売経験のある方大歓迎／早期キャリアアップも可能◆,人材コーディネーター,人材派遣・人材紹介・アウトソーシング,06-7777-2615,\n \n ■総合職（転勤の...,人材コーディネーター・カウンセラー,https://www.ecareer.ne.jp/positions/00047981001/5,大阪府,大阪市
4,4,株式会社スタッフサービス エンジニアリング事業本部（リクルートグループ）,\n \n 北海道・東北・北...,東京都千代田区神田練塀町85番地 ＪＥＢＬ秋葉原スクエア,正社員,CADオペレーター◆年間休日125日／残業月平均12時間／チームで助け合えるユニット制あり◆,CADオペレーター（電気・電子・半導体・機械・メカトロ）,その他IT関連,03-5209-7520,\n \n ■月給21万50...,機械・機構・金型・解析,https://www.ecareer.ne.jp/positions/0002142100...,東京都,千代田区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,野村不動産パートナーズ株式会社,\n \n 全国の当社拠点\...,東京都新宿区西新宿1-26-2新宿野村ビル21Ｆ,正社員,分譲マンションの管理◆野村不動産グループ／残業は20時間前後／年間休日123日◆,不動産管理,不動産仲介・不動産管理,NaN,"\n \n 月給215,00...",不動産関連,https://www.ecareer.ne.jp/positions/00046389001/3,東京都,新宿区
96,96,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,総合職◆賞与年2回／各種手当充実／年休120日／土日祝休み／面接1回（私服可）／最短5日で内定◆,設備保全,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 月給30万円以上...,施工管理・設備保守管理,https://www.ecareer.ne.jp/positions/00046654003/5,東京都,千代田区
97,97,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,事務◆志望動機や自己PRは白紙でOK！／最短5日で内定／賞与年2回／年休120日／土日祝休み◆,一般事務・庶務,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,一般事務・営業事務,https://www.ecareer.ne.jp/positions/00046654003/3,東京都,千代田区
98,98,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,点検・保守◆未経験歓迎／賞与年2回／年間休日120日／私服面接可／面接から入社まで最短5日◆,ビル・設備管理,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,施設・設備管理・警備・清掃,https://www.ecareer.ne.jp/positions/00046654003/1,東京都,千代田区


In [20]:
df_ka['給与情報実験']=df_ka['給与情報'].copy()
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('万円','0000円')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('千円','000円')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('万','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('(','あ')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace(')','あ')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace(',','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('：','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('.','')
df_ka['給与数字']=''
df_ka['給与区分']=''
df_ka['給与上限']=0
df_ka['給与下限']=float("inf")
df_ka['給なし'] = df_ka['給与情報実験'].str.match('(.*給.+円)', na=False)
df_ka['出来高']=df_ka['給与情報実験'].astype(str).str.extract('(出来高)')
df_ka['歩合']=df_ka['給与情報実験'].astype(str).str.extract('(歩合)')
df_ka['給あり']=df_ka['給与情報実験'].astype(str).str.extract('(.*給.+円)')
for i in range (len(df_ka['出来高'])):
    if df_ka['出来高'][i]=='出来高':
        df_ka['給与区分'][i]='出来高'
        df_ka['給与上限'][i]=df_ka['給与下限'][i]=np.nan
    else:
        if df_ka['給なし'][i]==False :
            n=str(df_ka['給与情報実験'][i])
            j=re.sub('あ.+?あ','', n)
            j=re.sub('[0-9]{1,2}時',"",j)
            j=re.sub('[0-9]{1,3}%|[0-9]{1,3}プラス|プラス[0-9]{1,3}|[0-9]{1,3}UP',"",j)
            j=re.sub('交通.+[0-9]+?円',"",j)
            df_ka['給与情報実験'][i]=j
#    m=re.findall(r'.給([0-9]+)円|～([0-9]+)円|([0-9]+)～円',df_ka['e'][i])
            m=re.findall(r'([0-9]{4,9})',df_ka['給与情報実験'][i])
            df_ka['給与数字'][i]=m
            h=re.findall(r'(.給)',df_ka['給与情報実験'][i])
            df_ka['給与区分'][i]=h
        if len(str(df_ka['給あり'][i]))!=0:
            n=str(df_ka['給あり'][i])
            j=re.sub('あ.+?あ','', n)
            j=re.sub('[0-9]{1,2}時',"",j)
            j=re.sub('[0-9]{1,3}%|[0-9]{1,3}プラス|プラス[0-9]{1,3}|[0-9]{1,3}UP',"",j)
            j=re.sub('交通.+[0-9]+?円',"",j)
            df_ka['給あり'][i]=j
#    m=re.findall(r'.給([0-9]+)円|～([0-9]+)円|([0-9]+)～円',df_ka['e'][i])
            m=re.findall(r'([0-9]{3,9}?)円',df_ka['給あり'][i])
            df_ka['給与数字'][i].extend(m)
            h=re.findall(r'(.給)',df_ka['給あり'][i])
            df_ka['給与区分'][i].extend(h)

for i in range(len(df_ka['給与区分'])):
    if  df_ka['出来高'][i]!='出来高' :
        flat =",".join(map(str, df_ka['給与区分'][i]))
        sp=flat.split(',')
        kore=set(sp)
        kore.discard('支給')
        kore.discard('合給')
        kore.discard('昇給')
        kore.discard('有給')
        kore.discard('、給')
        df_ka['給与区分'][i]=kore
        if df_ka['歩合'][i]=='歩合':
            df_ka['給与区分'][i].add('歩合')
        df_ka['給与区分'][i]=list(df_ka['給与区分'][i])
for i in range(len(df_ka['給与数字'])):
    if  df_ka['出来高'][i]!='出来高':
        for j in df_ka["給与数字"][i]:
            k=int(j)
            if k>df_ka['給与上限'][i]:
                df_ka['給与上限'][i]=k
            if k<df_ka['給与下限'][i]:
                df_ka['給与下限'][i]=k
  
   
for i in range(len(df_ka['歩合'])):
    if df_ka['歩合'][i]=='歩合':
        if df_ka['給与区分'][i][0]=='':
            df_ka['給与上限'][i]=df_ka['給与下限'][i]=np.nan
    else:
        if len(df_ka['給与区分'][i])==0:
            df_ka['給与区分']='日給'

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A valu

In [21]:
df_ka

,Unnamed: 0,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,...,市区町村,給与情報実験,給与数字,給与区分,給与上限,給与下限,給なし,出来高,歩合,給あり
0,0,株式会社ＥＣＯＰ,\n \n オフィス不要・ご...,京都府京都市西京区大枝沓掛町2-38,業務委託,窓フィルムの作業スタッフ◆未経験歓迎／切って貼るだけの簡単作業／加盟金・研修費0円／副業OK◆,設備工事スタッフ,清掃・メンテナンス,075-333-4407,\n \n 完全歩合制\n ...,...,京都市,\n \n 完全歩合制\n ...,"[30000, 5000000, 20000, 2000, 1200000, 100000,...","[, 歩合]",NaN,NaN,False,NaN,歩合,nan
1,1,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,営業職◆全国に勤務地あり／学歴・経験不問《年収4000万も可能》共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,...,福岡市,\n \n ■月給＋高歩合給...,"[270000, 890000, 51522, 136969, 10000, 5000, 1...","[月給, 歩合]",4630000.0,5000.0,False,NaN,歩合,月給270000円～890000円
2,2,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,省エネアドバイザー◆学歴・経験不問/年収4000万も可能/共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,...,福岡市,\n \n ■月給＋高歩合給...,"[270000, 890000, 51522, 136969, 10000, 5000, 1...","[月給, 歩合]",4630000.0,5000.0,False,NaN,歩合,月給270000円～890000円
3,3,株式会社ｏｒ ｓｔｙｌｅ,\n \n 勤務地：希望を考...,大阪府大阪市西区江戸堀2-1-1,正社員,介護に特化した人材サービス営業◆営業・販売経験のある方大歓迎／早期キャリアアップも可能◆,人材コーディネーター,人材派遣・人材紹介・アウトソーシング,06-7777-2615,\n \n ■総合職（転勤の...,...,大阪市,\n \n ■総合職（転勤の...,"[280000, 260000, 280000]",[月給],280000.0,260000.0,False,NaN,NaN,月給280000円
4,4,株式会社スタッフサービス エンジニアリング事業本部（リクルートグループ）,\n \n 北海道・東北・北...,東京都千代田区神田練塀町85番地 ＪＥＢＬ秋葉原スクエア,正社員,CADオペレーター◆年間休日125日／残業月平均12時間／チームで助け合えるユニット制あり◆,CADオペレーター（電気・電子・半導体・機械・メカトロ）,その他IT関連,03-5209-7520,\n \n ■月給21万50...,...,千代田区,\n \n ■月給21500...,"[215000, 235000, 7090000, 590000, 6030000, 503...",[月給],7090000.0,215000.0,False,NaN,NaN,■月給215000円
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,野村不動産パートナーズ株式会社,\n \n 全国の当社拠点\...,東京都新宿区西新宿1-26-2新宿野村ビル21Ｆ,正社員,分譲マンションの管理◆野村不動産グループ／残業は20時間前後／年間休日123日◆,不動産管理,不動産仲介・不動産管理,NaN,"\n \n 月給215,00...",...,新宿区,\n \n 月給215000...,"[215000, 4000000, 301500, 215000]",[月給],4000000.0,215000.0,False,NaN,NaN,月給215000円
96,96,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,総合職◆賞与年2回／各種手当充実／年休120日／土日祝休み／面接1回（私服可）／最短5日で内定◆,設備保全,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 月給30万円以上...,...,千代田区,\n \n 月給300000...,"[300000, 77000, 3000000, 5000000, 3200000, 270...",[月給],7900000.0,77000.0,False,NaN,NaN,月給300000円
97,97,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,事務◆志望動機や自己PRは白紙でOK！／最短5日で内定／賞与年2回／年休120日／土日祝休み◆,一般事務・庶務,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,...,千代田区,\n \n 東京・大阪月給2...,"[263000, 46000, 231000, 263000, 254000, 38000,...",[月給],7900000.0,38000.0,False,NaN,NaN,東京・大阪月給263000円
98,98,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,点検・保守◆未経験歓迎／賞与年2回／年間休日120日／私服面接可／面接から入社まで最短5日◆,ビル・設備管理,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,...,千代田区,\n \n 東京・大阪月給2...,"[263000, 46000, 231000, 263000, 254000, 38000,...",[月給],7900000.0,38000.0,False,NaN,NaN,東京・大阪月給263000円


In [22]:
df2=df_ka.drop(['Unnamed: 0','給なし', '出来高', '給あり', '給与情報実験', '給与数字','歩合'],axis=1)

In [23]:
df2

,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,仕事内容,ページurl,都道府県,市区町村,給与区分,給与上限,給与下限
0,株式会社ＥＣＯＰ,\n \n オフィス不要・ご...,京都府京都市西京区大枝沓掛町2-38,業務委託,窓フィルムの作業スタッフ◆未経験歓迎／切って貼るだけの簡単作業／加盟金・研修費0円／副業OK◆,設備工事スタッフ,清掃・メンテナンス,075-333-4407,\n \n 完全歩合制\n ...,工事スタッフ、その他建築・土木・設備工事関連,https://www.ecareer.ne.jp/positions/00046272001/4,京都府,京都市,"[, 歩合]",NaN,NaN
1,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,営業職◆全国に勤務地あり／学歴・経験不問《年収4000万も可能》共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/7,福岡県,福岡市,"[月給, 歩合]",4630000.0,5000.0
2,ＥＣＯ信頼サービス株式会社,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,正社員,省エネアドバイザー◆学歴・経験不問/年収4000万も可能/共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,営業・渉外・MR,https://www.ecareer.ne.jp/positions/00047852001/8,福岡県,福岡市,"[月給, 歩合]",4630000.0,5000.0
3,株式会社ｏｒ ｓｔｙｌｅ,\n \n 勤務地：希望を考...,大阪府大阪市西区江戸堀2-1-1,正社員,介護に特化した人材サービス営業◆営業・販売経験のある方大歓迎／早期キャリアアップも可能◆,人材コーディネーター,人材派遣・人材紹介・アウトソーシング,06-7777-2615,\n \n ■総合職（転勤の...,人材コーディネーター・カウンセラー,https://www.ecareer.ne.jp/positions/00047981001/5,大阪府,大阪市,[月給],280000.0,260000.0
4,株式会社スタッフサービス エンジニアリング事業本部（リクルートグループ）,\n \n 北海道・東北・北...,東京都千代田区神田練塀町85番地 ＪＥＢＬ秋葉原スクエア,正社員,CADオペレーター◆年間休日125日／残業月平均12時間／チームで助け合えるユニット制あり◆,CADオペレーター（電気・電子・半導体・機械・メカトロ）,その他IT関連,03-5209-7520,\n \n ■月給21万50...,機械・機構・金型・解析,https://www.ecareer.ne.jp/positions/0002142100...,東京都,千代田区,[月給],7090000.0,215000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,野村不動産パートナーズ株式会社,\n \n 全国の当社拠点\...,東京都新宿区西新宿1-26-2新宿野村ビル21Ｆ,正社員,分譲マンションの管理◆野村不動産グループ／残業は20時間前後／年間休日123日◆,不動産管理,不動産仲介・不動産管理,NaN,"\n \n 月給215,00...",不動産関連,https://www.ecareer.ne.jp/positions/00046389001/3,東京都,新宿区,[月給],4000000.0,215000.0
96,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,総合職◆賞与年2回／各種手当充実／年休120日／土日祝休み／面接1回（私服可）／最短5日で内定◆,設備保全,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 月給30万円以上...,施工管理・設備保守管理,https://www.ecareer.ne.jp/positions/00046654003/5,東京都,千代田区,[月給],7900000.0,77000.0
97,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,事務◆志望動機や自己PRは白紙でOK！／最短5日で内定／賞与年2回／年休120日／土日祝休み◆,一般事務・庶務,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,一般事務・営業事務,https://www.ecareer.ne.jp/positions/00046654003/3,東京都,千代田区,[月給],7900000.0,38000.0
98,株式会社夢真,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,正社員,点検・保守◆未経験歓迎／賞与年2回／年間休日120日／私服面接可／面接から入社まで最短5日◆,ビル・設備管理,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,施設・設備管理・警備・清掃,https://www.ecareer.ne.jp/positions/00046654003/1,東京都,千代田区,[月給],7900000.0,38000.0


In [24]:
dfrust=df2.reindex(columns=['法人名','施設名','勤務地住所','本社住所','都道府県','市区町村','雇用区分','ページタイトル','媒体記載職種','業種','電話番号','給与情報','給与区分','給与下限','給与上限','仕事内容','取得日','媒体名','派遣・紹介企業','ページurl'])

In [25]:
import datetime
dfrust['取得日']=datetime.date.today( )
dfrust['媒体名']='ecareer'

In [27]:
dfrust

,法人名,施設名,勤務地住所,本社住所,都道府県,市区町村,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,給与区分,給与下限,給与上限,仕事内容,取得日,媒体名,派遣・紹介企業,ページurl
0,株式会社ＥＣＯＰ,NaN,\n \n オフィス不要・ご...,京都府京都市西京区大枝沓掛町2-38,京都府,京都市,業務委託,窓フィルムの作業スタッフ◆未経験歓迎／切って貼るだけの簡単作業／加盟金・研修費0円／副業OK◆,設備工事スタッフ,清掃・メンテナンス,075-333-4407,\n \n 完全歩合制\n ...,"[, 歩合]",NaN,NaN,工事スタッフ、その他建築・土木・設備工事関連,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00046272001/4
1,ＥＣＯ信頼サービス株式会社,NaN,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,福岡県,福岡市,正社員,営業職◆全国に勤務地あり／学歴・経験不問《年収4000万も可能》共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,"[月給, 歩合]",5000.0,4630000.0,営業・渉外・MR,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00047852001/7
2,ＥＣＯ信頼サービス株式会社,NaN,\n \n 下記のいずれかの...,福岡県福岡市東区馬出1-17-1ＥＣＯ信頼ビル,福岡県,福岡市,正社員,省エネアドバイザー◆学歴・経験不問/年収4000万も可能/共に挑戦し、実現を果たそう◆,個人向けセールス・営業,電力・ガス・その他エネルギー,NaN,\n \n ■月給＋高歩合給...,"[月給, 歩合]",5000.0,4630000.0,営業・渉外・MR,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00047852001/8
3,株式会社ｏｒ ｓｔｙｌｅ,NaN,\n \n 勤務地：希望を考...,大阪府大阪市西区江戸堀2-1-1,大阪府,大阪市,正社員,介護に特化した人材サービス営業◆営業・販売経験のある方大歓迎／早期キャリアアップも可能◆,人材コーディネーター,人材派遣・人材紹介・アウトソーシング,06-7777-2615,\n \n ■総合職（転勤の...,[月給],260000.0,280000.0,人材コーディネーター・カウンセラー,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00047981001/5
4,株式会社スタッフサービス エンジニアリング事業本部（リクルートグループ）,NaN,\n \n 北海道・東北・北...,東京都千代田区神田練塀町85番地 ＪＥＢＬ秋葉原スクエア,東京都,千代田区,正社員,CADオペレーター◆年間休日125日／残業月平均12時間／チームで助け合えるユニット制あり◆,CADオペレーター（電気・電子・半導体・機械・メカトロ）,その他IT関連,03-5209-7520,\n \n ■月給21万50...,[月給],215000.0,7090000.0,機械・機構・金型・解析,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/0002142100...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,野村不動産パートナーズ株式会社,NaN,\n \n 全国の当社拠点\...,東京都新宿区西新宿1-26-2新宿野村ビル21Ｆ,東京都,新宿区,正社員,分譲マンションの管理◆野村不動産グループ／残業は20時間前後／年間休日123日◆,不動産管理,不動産仲介・不動産管理,NaN,"\n \n 月給215,00...",[月給],215000.0,4000000.0,不動産関連,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00046389001/3
96,株式会社夢真,NaN,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,東京都,千代田区,正社員,総合職◆賞与年2回／各種手当充実／年休120日／土日祝休み／面接1回（私服可）／最短5日で内定◆,設備保全,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 月給30万円以上...,[月給],77000.0,7900000.0,施工管理・設備保守管理,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00046654003/5
97,株式会社夢真,NaN,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,東京都,千代田区,正社員,事務◆志望動機や自己PRは白紙でOK！／最短5日で内定／賞与年2回／年休120日／土日祝休み◆,一般事務・庶務,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,[月給],38000.0,7900000.0,一般事務・営業事務,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00046654003/3
98,株式会社夢真,NaN,\n \n 全国いずれかのプ...,東京都千代田区丸の内1-4-1丸の内永楽ビルディング22F,東京都,千代田区,正社員,点検・保守◆未経験歓迎／賞与年2回／年間休日120日／私服面接可／面接から入社まで最短5日◆,ビル・設備管理,人材派遣・人材紹介・アウトソーシング,NaN,\n \n 東京・大阪：月給...,[月給],38000.0,7900000.0,施設・設備管理・警備・清掃,2020-12-17,ecareer,NaN,https://www.ecareer.ne.jp/positions/00046654003/1


In [28]:
dfrust.to_csv('ecareer_{}.csv'.format(datetime.date.today()))